In [1]:
import librosa
import pandas as pd
import numpy as np
import random
import shutil
import os

### 메타데이터 확인

In [2]:
meta_path = '/home/woongjae/noise-tracing/Datasets/VCTK-Corpus/meta_vctk.csv'
meta_df = pd.read_csv(meta_path, sep=',')
meta_df.head()

,path,Speaker name,Utterence name (file name),TTS or VC,Is multi-speaker?,Language,Noise type 1,Source link,utt,label,group,model,Gender,Age
0,./wav/p310_007.wav,p310,p310_007.wav,-,No,English,-,-,p310_007,bonafide,VCTK,-,Female,Adult
1,./wav/p236_263.wav,p236,p236_263.wav,-,No,English,-,-,p236_263,bonafide,VCTK,-,Female,Adult
2,./wav/p302_246.wav,p302,p302_246.wav,-,No,English,-,-,p302_246,bonafide,VCTK,-,Male,Adult
3,./wav/p333_423.wav,p333,p333_423.wav,-,No,English,-,-,p333_423,bonafide,VCTK,-,Female,Adult
4,./wav/p253_277.wav,p253,p253_277.wav,-,No,English,-,-,p253_277,bonafide,VCTK,-,Female,Adult


### 1초 이상의 파일들만 카운트

In [3]:
def calculate_duration(row):
    file_path = row['path']
    if os.path.exists(file_path):
        try:
            y, sr = librosa.load(file_path, sr=16000)  # 샘플링 레이트 16,000으로 고정
            return len(y) / sr  # 길이 계산
        except Exception as e:
            print(f"오류 발생: {file_path}, {e}")
            return 0
    else:
        return 0

In [6]:
base_path = '/home/woongjae/noise-tracing/Datasets/VCTK-Corpus/wav'
meta_df['path'] = meta_df['Utterence name (file name)'].apply(lambda x: os.path.join(base_path, x))

# 파일 길이 계산 및 필터링
meta_df['duration'] = meta_df.apply(calculate_duration, axis=1)
filtered_df = meta_df[meta_df['duration'] >= 1]  # 1초 이상인 파일만 필터링
filtered_out_count = len(meta_df) - len(filtered_df)  # 필터링된 파일 개수


# 각 스피커별 파일 개수 계산
speaker_file_counts = filtered_df.groupby("Speaker name").size()

# 결과 출력
print("1초 이상 파일만 필터링 후 스피커별 파일 개수:")
print(speaker_file_counts)
print(f"1초 미만으로 필터링된 파일 수: {filtered_out_count}")

# 필요 시 데이터프레임을 저장
output_path = '/home/woongjae/noise-tracing/new_dataset/vctk_speaker_file_counts.csv'
speaker_file_counts.to_csv(output_path, header=["File Count"])
print(f"스피커별 파일 개수를 {output_path}에 저장했습니다.")

1초 이상 파일만 필터링 후 스피커별 파일 개수:
Speaker name
p225    231
p226    356
p227    389
p228    366
p229    379
       ... 
p362    423
p363    418
p364    301
p374    424
p376    292
Length: 109, dtype: int64
1초 미만으로 필터링된 파일 수: 4
스피커별 파일 개수를 /home/woongjae/noise-tracing/new_dataset/vctk_speaker_file_counts.csv에 저장했습니다.


### 각 화자별로 100개씩 랜덤으로 뽑기

In [4]:
meta_path = '/home/woongjae/noise-tracing/Datasets/VCTK-Corpus/meta_vctk.csv'
meta_df = pd.read_csv(meta_path, sep=',')

output_dir = '/home/woongjae/noise-tracing/new_dataset/Dataset/Bonafide/Clean'  # 새 폴더
os.makedirs(output_dir, exist_ok=True)

base_dir = '/home/woongjae/noise-tracing/Datasets/VCTK-Corpus'

# 새로운 메타데이터 저장용 리스트
new_meta = []

# 스피커 목록 불러오기 (100개 이상 보유한 스피커만)
speaker_counts = meta_df['Speaker name'].value_counts()
eligible_speakers = speaker_counts[speaker_counts >= 100].index.tolist()

for speaker in eligible_speakers:
    speaker_df = meta_df[meta_df['Speaker name'] == speaker]
    sampled_df = speaker_df.sample(n=100, random_state=42)  # 랜덤 샘플링

    for _, row in sampled_df.iterrows():
        rel_path = row['path'].lstrip('./')
        abs_path = os.path.join(base_dir, rel_path)  # 전체 경로
        dst = os.path.join(output_dir, os.path.basename(rel_path))

        # 파일 복사
        shutil.copy(abs_path, dst)

        new_meta.append({
            'File_path': dst,
            'utt': row['utt'],
            'speaker ID': row['Speaker name'],
            'gender': row['Gender'],
            'Group': 'VCTK',
            'Label': 'bonafide'
        })

# 새 메타데이터 저장
new_df = pd.DataFrame(new_meta)
new_df.to_csv('meta_new_VCTK.csv', index=False)